### Instagram API

In [ ]:
client_id = ''
client_secret = ''

from instagram.client import InstagramAPI

api = InstagramAPI(client_id=client_id, client_secret=client_secret)

### Instagram Tag Count

In [ ]:
# get ig tag info -> how many IG posts have the given hashtag
used_tag = 'itpnyu'
num_media_tag = api.tag(used_tag).media_count
print num_media_tag

In [ ]:
# get ig tag info
used_tag = 'GOPdebate'
num_media_tag = api.tag(used_tag).media_count
print num_media_tag

In [ ]:
# get ig tag info
used_tag = 'IStandWithAhmed'
num_media_tag = api.tag(used_tag).media_count
print num_media_tag

### Instagram Media

In [ ]:
# get ig media based on tag

max_tag_id = 0
media_per_query = 33
ans = api.tag_recent_media(media_per_query, max_tag_id, used_tag)
ans

In [ ]:
print len(ans[0])

In [ ]:
print ans[1]

In [ ]:
# a more intuitive way to store the results is by using multiple variables

ans_media, next_ = api.tag_recent_media(media_per_query, max_tag_id, used_tag)

In [ ]:
print next_

In [ ]:
# attributes that our IG media object has:

all_media = []

for m in ans_media:
    print m.created_time
    print m.id
    print m.user
    print m.link
    print ''
    all_media.append(m)

In [ ]:
ans[1]

In [ ]:
from urlparse import urlparse

parsed = urlparse(next_)
print parsed
print parsed.query
print ''
print {a:b for a,b in [x.split('=') for x in parsed.query.split('&')]}

In [ ]:
# lets make a function out of this -> parse out the max_tag_id parameter

def extract_max_tag_id(txt):
    parsed = urlparse(txt)
    params = {a:b for a,b in [x.split('=') for x in parsed.query.split('&')]}
    return params['max_tag_id']


In [ ]:
max_tag_id = 0
media_per_query = 33

all_media, next_ = api.tag_recent_media(media_per_query, max_tag_id, used_tag)

while next_:
    more_media, next_ = api.tag_recent_media(count=media_per_query, tag_name=used_tag, with_next_url=next_)
    all_media.extend(more_media)
        
    print len(all_media)

    # get next max_tag_id
    #max_tag_id = extract_max_tag_id(ans[1])
    

In [ ]:
len(all_media)

In [ ]:
import pickle
path = '/class/itpmssd/datasets/'

pickle.dump(all_media, open(path+'%s_ig.p' % used_tag,'wb'))

In [ ]:
import pickle
path = '/class/itpmssd/datasets/'

all_media = pickle.load(open(path+'%s_ig.p' % used_tag,'rb'))

### Let's look at our data

In [ ]:
# returns a list of valid attributes for the given object -> IG media object / class

dir(all_media[0])

In [ ]:
# users
users = set([m.user.username for m in all_media])
len(users)

In [ ]:
# Show ten most liked IG posts

for m in sorted(all_media, key=lambda x: -x.like_count)[:10]:
    print m.like_count, m.caption, m.link, '\n'

In [ ]:
# comments - 10 most commented on post

for m in sorted(all_media, key=lambda x: -x.comment_count)[:10]:
    print m.like_count, m.caption, m.link, '\n'

In [ ]:
# comments - most commented on post

for m in sorted(all_media, key=lambda x: -x.like_count)[:20]:
    print m.like_count, m.comment_count, m.link

In [ ]:
# use pandas - plot over time
import pandas as pd

df = pd.DataFrame({'times':[m.created_time for m in all_media], 'users':[m.user.username for m in all_media], 'posts':[m.link for m in all_media]})

In [ ]:
def make_date(d):
    d = d.replace(second=0, minute=0)
    return d

df['dt']=df[['times']].apply(lambda x: make_date(x['times']), axis=1)

In [ ]:
df.set_index('dt')

In [ ]:
%pylab inline

df.groupby(df.dt).size().plot(figsize=(20,6))

In [ ]:
df[-10:]

In [ ]:
df.groupby(df.dt).size()['2015-09-15':].plot(figsize=(20,6))

In [ ]:
%pylab inline

l = [m.like_count for m in all_media]
c = [m.comment_count for m in all_media]

# let's plot the relationship between these two variables

scatter(l,c,s=50,alpha=.1)
title('likes vs. comments')
ylabel('comment count')
xlabel('like count')


In [ ]:
l_log = [log(m.like_count) for m in all_media]
c_log = [log(m.comment_count) for m in all_media]

# let's plot the relationship between these two variables
rcParams['figure.figsize'] = 12,6
scatter(l_log,c_log,s=50,alpha=.1)
title('likes vs. comments')
ylabel('comment count')
xlabel('like count')


### tags

In [ ]:
from collections import Counter

#all_tags = [t.name for t in m.tags for m in all_media if hasattr(m, 'tags')]

In [ ]:
all_tags = []
for m in all_media:
    if hasattr(m, 'tags'):
        all_tags += [t.name for t in m.tags]

In [ ]:
counted_tags = Counter(all_tags)
counted_tags.most_common(40)

### location

In [ ]:
# check if the object - m - has a location attribute
location_names = set([m.location.name for m in all_media if hasattr(m, 'location')])
len(location_names)
location_names

In [ ]:
locations = {m.user.username:m.location for m in all_media if hasattr(m, 'location')}

In [ ]:
for u,loc in locations.items()[:10]:
    print u,loc

In [ ]:
user_loc = locations['kashie333']

print user_loc.id
print user_loc.name
print user_loc.point.latitude
print user_loc.point.longitude

In [ ]:
# pip install geopy
# reverse geo-lookup -> for more geographic information
import json
from geopy.geocoders import Nominatim
geolocator = Nominatim()

location = geolocator.reverse((user_loc.point.latitude, user_loc.point.longitude))
print location.address
print json.dumps(location.raw, indent=1)
